In [ ]:
from __future__ import print_function
import os
import time
import logging
import argparse
import numpy as np
from visdom import Visdom
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from utils import *

# Teacher models:
# VGG11/VGG13/VGG16/VGG19, GoogLeNet, AlxNet, ResNet18, ResNet34, 
# ResNet50, ResNet101, ResNet152, ResNeXt29_2x64d, ResNeXt29_4x64d, 
# ResNeXt29_8x64d, ResNeXt29_32x64d, PreActResNet18, PreActResNet34, 
# PreActResNet50, PreActResNet101, PreActResNet152, 
# DenseNet121, DenseNet161, DenseNet169, DenseNet201, 
import models

# Student models:
# myNet, LeNet, FitNet

start_time = time.time()
# os.makedirs('./checkpoint', exist_ok=True)

# Training settings
parser = argparse.ArgumentParser(description='PyTorch DML 3S')

parser.add_argument('--dataset',
                    choices=['CIFAR10',
                             'CIFAR100'
                            ],
                    default='CIFAR10')
parser.add_argument('--net1',
                    choices=['ResNet8',
                             'ResNet15',
                             'ResNet16',
                             'ResNet20',
                             'ResNet32',
                             'ResNet50',
                             'ResNet56',
                             'ResNet110'
                            ],
                    default='ResNet20')
parser.add_argument('--net2',
                    choices=['ResNet8',
                             'ResNet15',
                             'ResNet16',
                             'ResNet20',
                             'ResNet32',
                             'ResNet50',
                             'ResNet56',
                             'ResNet110'
                            ],
                    default='ResNet56')

parser.add_argument('--net3',
                    choices=['ResNet8',
                             'ResNet15',
                             'ResNet16',
                             'ResNet20',
                             'ResNet32',
                             'ResNet50',
                             'ResNet56',
                             'ResNet110'
                            ],
                    default='ResNet110')

parser.add_argument('--n_class', type=int, default=10, metavar='N', help='num of classes')
parser.add_argument('--batch_size', type=int, default=128, metavar='N', help='input batch size for training')
parser.add_argument('--test_batch_size', type=int, default=128, metavar='N', help='input test batch size for training')
parser.add_argument('--epochs', type=int, default=20, metavar='N', help='number of epochs to train (default: 20)')
parser.add_argument('--lr', type=float, default=0.1, metavar='LR', help='learning rate (default: 0.01)')
parser.add_argument('--momentum', type=float, default=0.9, metavar='M', help='SGD momentum (default: 0.5)')
parser.add_argument('--device', default='cuda:1', type=str, help='device: cuda or cpu')
parser.add_argument('--print_freq', type=int, default=40, metavar='N', help='how many batches to wait before logging training status')

config = ['--dataset', 'CIFAR10', '--epochs', '200', '--n_class', '100', '--net1', 'ResNet110', '--net2', 'ResNet32', '--net3', 'ResNet16', '--device', 'cuda:1']
args = parser.parse_args(config)

device = args.device if torch.cuda.is_available() else 'cpu'
save_dir = './checkpoint/' + args.dataset + '/'

# models
net1 = getattr(models, args.net1)(num_classes=args.n_class)
net1.to(device)
net2 = getattr(models, args.net2)(num_classes=args.n_class)
net2.to(device)
net3 = getattr(models, args.net3)(num_classes=args.n_class)
net3.to(device)

# logging
logfile = save_dir + 'DML_3S_.log'
if os.path.exists(logfile):
    os.remove(logfile)
def log_out(info):
    f = open(logfile, mode='a')
    f.write(info)
    f.write('\n')
    f.close()
    print(info)
    
# visualizer
vis = Visdom(env='distill')
loss_win = vis.line(
    X=np.column_stack((0,0,0)),
    Y=np.column_stack((0,0,0)),
    opts=dict(
        title='DML 3S Loss',
        xlabel='epoch',
        xtickmin=0,
        ylabel='loss',
        ytickmin=0,
        ytickstep=0.5,
        legend=['net1_loss', 'net2_loss', 'net3_loss']
    ),
    name="loss"
)

acc_win = vis.line(
    X=np.column_stack((0,0,0)),
    Y=np.column_stack((0,0,0)),
    opts=dict(
        title='DML 3S Acc',
        xlabel='epoch',
        xtickmin=0,
        ylabel='accuracy',
        ytickmin=0,
        ytickmax=100,
        legend=['net1_acc', 'net2_acc', 'net3_acc']
    ),
    name="acc"
)


# data
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, 4),
    transforms.ToTensor(),
    normalize,
])
test_transform = transforms.Compose([transforms.ToTensor(), normalize])
train_set = getattr(datasets, args.dataset)(root='../data', train=True, download=True, transform=train_transform)
test_set = getattr(datasets, args.dataset)(root='../data', train=False, download=False, transform=test_transform)
train_loader = DataLoader(train_set, batch_size=args.batch_size, shuffle=True)
test_loader = DataLoader(test_set, batch_size=args.test_batch_size, shuffle=False)

# optimizer = optim.SGD(st_model.parameters(), lr=args.lr, momentum=args.momentum)
optimizer_1 = optim.SGD(net1.parameters(), lr=args.lr, momentum=0.9, weight_decay=5e-4)
optimizer_2 = optim.SGD(net2.parameters(), lr=args.lr, momentum=0.9, weight_decay=5e-4)
optimizer_3 = optim.SGD(net3.parameters(), lr=args.lr, momentum=0.9, weight_decay=5e-4)
lr_scheduler_1 = optim.lr_scheduler.MultiStepLR(optimizer_1, milestones=[100, 150])
lr_scheduler_2 = optim.lr_scheduler.MultiStepLR(optimizer_2, milestones=[100, 150])
lr_scheduler_3 = optim.lr_scheduler.MultiStepLR(optimizer_3, milestones=[100, 150])


# train with multi-teacher
def train(epoch, net1, net2, net3):
    print('Training:')
    # switch to train mode
    net1.train()
    net2.train()
    net3.train()
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses_1 = AverageMeter()
    losses_2 = AverageMeter()
    losses_3 = AverageMeter()
    top1_1 = AverageMeter()
    top1_2 = AverageMeter()
    top1_3 = AverageMeter()
    
    end = time.time()
    for i, (input, target) in enumerate(train_loader):

        # measure data loading time
        data_time.update(time.time() - end)

        input, target = input.to(device), target.to(device)
        
        # compute outputs
        _,_,_,_, output_1 = net1(input)
        _,_,_,_, output_2 = net2(input)
        _,_,_,_, output_3 = net3(input)
        logits_1 = F.softmax(output_1)
        logits_2 = F.softmax(output_2)
        logits_3 = F.softmax(output_3)
        
        optimizer_1.zero_grad()
        loss_1 = 0.5 * (nn.KLDivLoss()(logits_2, logits_1) + nn.KLDivLoss()(logits_3, logits_1)) + F.cross_entropy(output_1, target)
        loss_1.backward(retain_graph=True)
        optimizer_1.step()
        
        optimizer_2.zero_grad()
        loss_2 = 0.5 * (nn.KLDivLoss()(logits_1, logits_2) + nn.KLDivLoss()(logits_3, logits_2)) + F.cross_entropy(output_2, target)
        loss_2.backward(retain_graph=True)
        optimizer_2.step()
        
        optimizer_3.zero_grad()
        loss_3 = 0.5 * (nn.KLDivLoss()(logits_1, logits_3) + nn.KLDivLoss()(logits_2, logits_3)) + F.cross_entropy(output_3, target)
        loss_3.backward(retain_graph=True)
        optimizer_3.step()

        # measure accuracy and record loss
        net1_acc = accuracy(output_1, target)[0]
        net2_acc = accuracy(output_2, target)[0]
        net3_acc = accuracy(output_3, target)[0]
        losses_1.update(loss_1.item(), input.size(0))
        top1_1.update(net1_acc, input.size(0))
        losses_2.update(loss_2.item(), input.size(0))
        top1_2.update(net2_acc, input.size(0))
        losses_3.update(loss_3.item(), input.size(0))
        top1_3.update(net3_acc, input.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % args.print_freq == 0:
            log_out('[{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec@1 ({top1_1.avg:.3f})'.format(
                      i, len(train_loader), batch_time=batch_time,
                      data_time=data_time, loss=losses_1, top1_1=top1_1))
    return losses_1.avg, losses_2.avg, losses_3.avg, net1_acc.cpu().numpy(), net2_acc.cpu().numpy(), net3_acc.cpu().numpy()


def test(model):
    print('Testing:')
    # switch to evaluate mode
    model.eval()
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    end = time.time()
    with torch.no_grad():
        for i, (input, target) in enumerate(test_loader):
            input, target = input.to(device), target.to(device)

            # compute output
            _,_,_,_,output = model(input)
            loss = F.cross_entropy(output, target)

            output = output.float()
            loss = loss.float()

            # measure accuracy and record loss
            test_acc = accuracy(output.data, target.data)[0]
            losses.update(loss.item(), input.size(0))
            top1.update(test_acc, input.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % args.print_freq == 0:
                log_out('Test: [{0}/{1}]\t'
                      'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                      'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                      'Prec@1 {top1.val:.3f} ({top1.avg:.3f})'.format(
                          i, len(test_loader), batch_time=batch_time, loss=losses,
                          top1=top1))

    log_out(' * {0} Prec@1 {top1.avg:.3f}'.format(model.model_name, top1=top1))

    return losses.avg, test_acc.cpu().numpy(), top1.avg.cpu().numpy()


print('*-----------------DML----------------*')
best_acc = 0
for epoch in range(1, args.epochs + 1):
    log_out("\n===> epoch: {}/{}".format(epoch, args.epochs))
#     log_out('current lr {:.5e}'.format(optimizer_1.param_groups[0]['lr']))
    lr_scheduler_1.step()
    lr_scheduler_2.step()
    lr_scheduler_3.step()
    train_loss_1, train_loss_2, train_loss_3, net1_acc, net2_acc, net3_acc = train(epoch, net1, net2, net3)
    # visaulize loss
    vis.line(np.column_stack((train_loss_1, train_loss_2, train_loss_3)), np.column_stack((epoch, epoch, epoch)), loss_win, update="append")
    _, test_acc_1, top1_1 = test(net1)
    _, test_acc_2, top1_2 = test(net2)
    _, test_acc_3, top1_3 = test(net3)
    vis.line(np.column_stack((top1_1, top1_2, top1_3)), np.column_stack((epoch, epoch, epoch)), acc_win, update="append")
    best_acc = max(top1_1, top1_2, top1_3, best_acc)

log_out("@ BEST Prec: {:.4f}".format(best_acc))
log_out("--- {:.3f} mins ---".format((time.time() - start_time)/60))


/home/data/yaliu/jupyterbooks/multi-KD/models/teacher/resnet20.py:36: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  init.kaiming_normal(m.weight)
/home/data/yaliu/jupyterbooks/multi-KD/models/student/resnet_s.py:13: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  init.kaiming_normal(m.weight)


Files already downloaded and verified
*-----------------DML----------------*

===> epoch: 1/200
Training:


/home/yaliu/Dev/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:192: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/yaliu/Dev/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:193: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/yaliu/Dev/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:194: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/yaliu/Dev/anaconda3/envs/py37/lib/python3.7/site-packages/torch/nn/functional.py:1992: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  

[0/391]	Time 0.690 (0.690)	Data 0.036 (0.036)	Loss 8.4445 (8.4445)	Prec@1 (0.000)
[40/391]	Time 0.269 (0.280)	Data 0.018 (0.019)	Loss 2.0694 (3.1600)	Prec@1 (14.634)
[80/391]	Time 0.270 (0.275)	Data 0.018 (0.019)	Loss 1.9104 (2.5826)	Prec@1 (19.502)
[120/391]	Time 0.271 (0.273)	Data 0.019 (0.019)	Loss 1.7678 (2.3481)	Prec@1 (22.546)
[160/391]	Time 0.270 (0.273)	Data 0.018 (0.019)	Loss 1.6953 (2.2107)	Prec@1 (24.845)
[200/391]	Time 0.274 (0.273)	Data 0.019 (0.019)	Loss 1.6655 (2.1096)	Prec@1 (26.959)
[240/391]	Time 0.273 (0.273)	Data 0.019 (0.019)	Loss 1.6604 (2.0331)	Prec@1 (28.524)
[280/391]	Time 0.273 (0.273)	Data 0.018 (0.019)	Loss 1.8898 (1.9754)	Prec@1 (29.896)
[320/391]	Time 0.274 (0.273)	Data 0.019 (0.019)	Loss 1.4203 (1.9264)	Prec@1 (31.128)
[360/391]	Time 0.275 (0.273)	Data 0.019 (0.019)	Loss 1.4795 (1.8835)	Prec@1 (32.354)
Testing:
Test: [0/79]	Time 0.036 (0.036)	Loss 1.6694 (1.6694)	Prec@1 36.719 (36.719)
Test: [40/79]	Time 0.035 (0.035)	Loss 1.6807 (1.5549)	Prec@1 39.844 (4

 * ResNet110 Prec@1 63.470
Testing:
Test: [0/79]	Time 0.022 (0.022)	Loss 0.7138 (0.7138)	Prec@1 75.781 (75.781)
Test: [40/79]	Time 0.022 (0.022)	Loss 0.7712 (0.7154)	Prec@1 72.656 (75.877)
 * ResNet32 Prec@1 76.150
Testing:
Test: [0/79]	Time 0.020 (0.020)	Loss 0.9502 (0.9502)	Prec@1 67.969 (67.969)
Test: [40/79]	Time 0.019 (0.019)	Loss 1.1082 (1.0371)	Prec@1 62.500 (65.396)
 * ResNet16 Prec@1 65.110

===> epoch: 7/200
Training:
[0/391]	Time 0.311 (0.311)	Data 0.021 (0.021)	Loss 0.7773 (0.7773)	Prec@1 (71.094)
[40/391]	Time 0.305 (0.303)	Data 0.021 (0.020)	Loss 0.6742 (0.7162)	Prec@1 (74.486)
[80/391]	Time 0.297 (0.303)	Data 0.019 (0.020)	Loss 0.7383 (0.7163)	Prec@1 (74.701)
[120/391]	Time 0.309 (0.303)	Data 0.020 (0.020)	Loss 0.5117 (0.7097)	Prec@1 (74.942)
[160/391]	Time 0.298 (0.303)	Data 0.020 (0.020)	Loss 0.6782 (0.7077)	Prec@1 (75.019)
[200/391]	Time 0.312 (0.307)	Data 0.020 (0.020)	Loss 0.7107 (0.7041)	Prec@1 (75.124)
[240/391]	Time 0.309 (0.308)	Data 0.020 (0.020)	Loss 0.6888 (0